In [ ]:
from datetime import datetime, timedelta
import os
import openpyxl
import os
import pandas as pd

ROOT_DIR = './'

MARKS = ['СКС', "СКД", "ДСТ", "СБС","ДССК"]
transports = ['all', 'auto', 'wagon', 'container']
VL06_TRANSPORT_TYPES = {'auto':'ZAUT', 'wagon':'ZRLC','container':'?'}
VSK_ID = 9021
print("Ready!")

In [ ]:
#Работаем с датами
cur_week = []
next_week = []
#Ввод первого дня недели
first_day_of_week = datetime(2020, 11, 16, 0, 0)
day = timedelta(days = 1)
                                
                
for i in range(7):
    cur_week.append(first_day_of_week + day*i)
    
first_day_of_next_week = cur_week[-1] + day
for i in range(7):
    next_week.append(first_day_of_next_week + day*i)

last_day_of_week = cur_week[-1]
#все дни от 1 дня месяца до последнего дня отчётной недели
nakop_dates = []
tmp_date = last_day_of_week
while tmp_date.month == last_day_of_week.month:
    nakop_dates.append(tmp_date)
    tmp_date -= day

nakop_dates.reverse()    
print('Накопленные дни',nakop_dates)
print("Текущая неделя",cur_week)
print("СЛедующая неделя",next_week)

def check_month(week, next_week):
    cur_month = first_day_of_week.month
    for d in week:
        if d.month > cur_month:
            return ['cur_week', week.index(d)]
    
    for d in next_week:
        if d.month > cur_month:
            return ['next_week', next_week.index(d)]

def divide_by_month(week, next_week):
    tmp = check_month(week, next_week)
    this_month = []
    next_month = []
    if tmp[0] == 'cur_week':
        this_month.extend(week[:tmp[1]])
        next_month.extend(week[tmp[1]:])
        next_month.extend(next_week)
    if tmp[0] == 'next_week':
        this_month.extend(week)
        this_month.extend(next_week[:tmp[1]])
        next_month.extend(next_week[tmp[1]:])
    return this_month, next_month

#day_in_this_month, days_in_next_month = divide_by_month(cur_week, next_week)

In [ ]:
#Загрузка образца отчёта
wb = openpyxl.load_workbook('weekly_basis.xlsx')
report_sheet = wb['Форма горизонталь1']


report_titles = report_sheet['A1':'AL3']
hiden_data = report_sheet['A4':'G8']
rows = {'sks':report_sheet['G4':'AL4'],'skd':report_sheet['G5':'AL5'],'tep35':report_sheet['G6':'AL6'],
        'tep50':report_sheet['G7':'AL7'],'total':report_sheet['G8':'AL8']}




In [ ]:
#Подгрузка отчётов из SAP 
#main =  pd.read_excel(os.path.join(ROOT_DIR, 'weekly_basis.xlsx'),sheet_name = 'MD maxV2')
#добавить проверку на верность даты начала и конца выгрузки
prod_report  = pd.read_excel(os.path.join(ROOT_DIR, 'ZPP_PROD_REPORT16-22.XLSX'),sheet_name = 'Sheet1', thousands = ',').drop(columns='Продукт').rename(columns={'Продукт.1':'Продукт'})
mb52_report  = pd.read_excel(os.path.join(ROOT_DIR, 'MB52.XLSX'),sheet_name = 'Sheet1', thousands = ',')

orion_df = pd.read_excel(os.path.join(ROOT_DIR, 'orion.XLSX'),sheet_name = 'ОАО Воронежсинтезкаучук', skiprows=2, thousands = ',')
#orion_next_df = pd.read_excel(os.path.join(ROOT_DIR, 'ORION_oct.XLSX'),sheet_name = 'ОАО Воронежсинтезкаучук', skiprows=2, thousands = ',')

VL06 = pd.read_excel(os.path.join(ROOT_DIR, 'VK06-26.11.XLSX'),sheet_name = 'Sheet1')[['Поставка','Название', 'Завод', 'Объем поставки', 'Ид. ТранспСредства', 'Дата Накладной', 'Вид ТранспСредства']]
    
main =  pd.read_excel(os.path.join(ROOT_DIR, 'weekly_basis.xlsx'),sheet_name = 'MD maxV2')
orion_df

In [ ]:
#Используемые функции(отлаженные и протестированные)
#печатает слайс
def print_row(row_slice):
    for cell_obj in row_slice:
        for cell in cell_obj:
            print(cell.value, cell.column, cell.coordinate)
#создает эксель, в который пишутся результаты
def create_resulting_wb():
    wb_result = openpyxl.Workbook()
    wb_result.create_sheet(title = 'Отчёт', index = 0)
    result_sheet = wb_result['Отчёт']
    for irow in range(8):
        result_sheet.append([])
    wb_result.save('weekly_report.xlsx')
    return wb_result
#копирую инфу из одного экселя в другой
def copy_cells(slice_to_copy,sheet_we_copy_in):
    for cells_obj in slice_to_copy:
        for cell in cells_obj:
            cell_tmp = sheet_we_copy_in.cell(row = cell.row, column = cell.column)
            cell_tmp.value = cell.value
    wb_result.save('weekly_report.xlsx')
    #wb_test.save('weekly_report.xlsx')
    
    
#Посчитать последнюю строку    
#Вызывать последней
def find_total(rep_sheet):
    total_positions = []
    for cell_obj in rows['total']:
        for cell in cell_obj:
            if type(cell.value) == str:
                total_positions.append((cell.value, cell.column, cell.coordinate))
    for i in total_positions:
        try:
            rep_sheet[i[2]] = (
                                  float(rep_sheet.cell(column=i[1],row=4).value) + 
                                  float(rep_sheet.cell(column=i[1],row=5).value) + 
                                  float(rep_sheet.cell(column=i[1],row=6).value) + 
                                  float(rep_sheet.cell(column=i[1],row=7).value)
                                  )
        except ValueError:
            #print(i[2])
            rep_sheet[i[2]] = ''
    #return rep_sheet['G8':'AL8']

def put_info_in_col(col_n, data_dict, sheet_we_put_in):
    cell_tmp = sheet_we_put_in.cell(row = 4, column = col_n)#CKC
    cell_tmp.value =data_dict['СКС']
    cell_tmp = sheet_we_put_in.cell(row = 5, column = col_n)#CKД+ДССК+СКД-НД
    cell_tmp.value =data_dict['СКД']
    cell_tmp = sheet_we_put_in.cell(row = 6, column = col_n)#ДСТ
    cell_tmp.value =data_dict['ДСТ']
    cell_tmp = sheet_we_put_in.cell(row = 7, column = col_n)#CБС
    cell_tmp.value =data_dict['СБС']
    wb_result.save('weekly_report.xlsx')


#Получаем данные для 17стобца из ZPP_PROD_REPORT -> вариант VSK_PERIOD_PRD (Отчёт ВСК производства и сортировки продукции. Паспортизация)
#вызывать предпоследней
def get_column_17_info(df_zpp_prod_report):
    #['СКС', "СКД"+"ДССК", "ДСТ", "СБС"]
    RESULT = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    for i in range(prod_report.shape[0]):
        row_tmp = prod_report.iloc[i,:]
        row_tmp['Продукт'] = str(row_tmp['Продукт'])
        if MARKS[0] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#CKC - верно
            RESULT['СКС'] += row_tmp['Паспортизация'].astype(float) 
            continue
        if MARKS[1] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#СКД - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[4] in row_tmp['Продукт'] and not 'ЭП' in row_tmp['Продукт']:#ДССК - верно
            RESULT['СКД'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[2] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#ДСТ
            RESULT['ДСТ'] += row_tmp['Паспортизация'].astype(float)
            continue
        if MARKS[3] in row_tmp['Продукт']:# and not 'ЭП' in row_tmp['Продукт']:#СБС
            RESULT['СБС'] += row_tmp['Паспортизация'].astype(float) 
            continue
    return RESULT

def get_sum_of_series(s):
    sym = 0
    for i in s.values:
        sym += i
    return sym

#Получаем данные для 10стобца из MB52 -> вариант BOGDANOVA
#@Depricated
# def get_column10_info(df_mb52_report):
#     RESULT_CLEAR = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
#     RESULT_BRAK611 = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
#     RESULT_EP = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0,'ЭП-ТЭП':0}

#    #готовый продукт без ЭП и брака 611
#     for i in MARKS:
#         #1 без брака и ЭП
#         if i == 'ДССК':
#             RESULT_CLEAR['СКД'] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#         else:
#             if(i == 'СБС') or (i =='ДСТ'):
#                 RESULT_CLEAR[i] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True)  & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#             else:
#                 RESULT_CLEAR[i] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == False)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())
#        #2 БРАК611
#         if i == 'ДССК':
#             RESULT_BRAK611['СКД'] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#         else:
#             if(i == 'СБС') or (i =='ДСТ'):
#                 RESULT_BRAK611[i] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True)  & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#             else:
#                 RESULT_BRAK611[i] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains(i) == True) & (df_mb52_report['Краткий текст материала'].str.contains(' 611') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())

#     RESULT_EP['СКД'] +=get_sum_of_series( df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ДССК') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#         #Как отличить ЭП ДСТ от ЭП СБС?
#     RESULT_EP['ЭП-ТЭП'] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ТЭП') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
#     RESULT_EP['СКС'] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-СКС') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())
#     RESULT_EP['СКД'] += get_sum_of_series(df_mb52_report[(df_mb52_report['Краткий текст материала'].str.contains('ЭП-ПБ') == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())

#     #return RESULT_CLEAR, RESULT_BRAK611, RESULT_EP
#     #return 'Продукт:'+str(RESULT_CLEAR), 'БРАК:'+ str(RESULT_BRAK611), 'ЭП:'+str(RESULT_EP)
#     #{'СКС':clear['СКС'] + brak['СКС'], 'СКД':clear['СКД'] + brak['СКД'], 'ДСТ':clear['ДСТ'] + brak['ДСТ'], 'СБС':clear['СБС'] + brak['СБС']}
#     return {'СКС':RESULT_CLEAR['СКС'] + RESULT_BRAK611['СКС'] + RESULT_EP['СКС'], 'СКД':RESULT_CLEAR['СКД'] + RESULT_BRAK611['СКД'] + RESULT_EP['СКД'],
#             'ДСТ':RESULT_CLEAR['ДСТ'] + RESULT_BRAK611['ДСТ'] + RESULT_EP['ДСТ'], 'СБС':RESULT_CLEAR['СБС'] + RESULT_BRAK611['СБС'] + RESULT_EP['СБС']}

    

def get_mb52():
    RESULT = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    for i in MARKS:
        if i == 'ДССК':
            RESULT['СКД'] += get_sum_of_series(mb52_report[(mb52_report['Краткий текст материала'].str.contains(i) == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
        else:
            RESULT[i] += get_sum_of_series(mb52_report[(mb52_report['Краткий текст материала'].str.contains(i) == True)][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())#.reset_index().drop(columns = 'index')
    ep_tep = mb52_report[(mb52_report['Краткий текст материала'].str.contains('ЭП-ТЭП') == True)][['Внешний номер партии MES','СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']]
    RESULT['ДСТ'] += get_sum_of_series(ep_tep[ep_tep['Внешний номер партии MES'].str.contains('9420') == True][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())
    RESULT['СБС'] += get_sum_of_series(ep_tep[ep_tep['Внешний номер партии MES'].str.contains('9520') == True][['СвобИспользЗапас','На КонтрКачества', 'ЗапОгрИспользования', 'Блокированный запас']].sum())
    return RESULT
        
        #ЭП-ТЭП (Внешний -> 9520 СБС, 9420 ДСТ)
#Считаем все столбцы под формулами
#11,12,18,19,20,21,24,,27,30,33,34
def create_formulas_cols(resulting_sheet):
    #[[куда поместить результат, оператор1, действие, оператор2, оператор3]-  действие одно на все операторы
    formulas= [[11, 10, '/',8], [12, 13, '+',14, 15], [18, 17, '/',16], [19, 28, '+',31, 25], [20, 26, '+', 29, 32], 
              [21, 20, '/',19], [24, 23, '/',22],[27, 26, '/',25], [30, 29, '/', 28], [33, 32, '/', 31], [34, 23, '-',22]]
    for i in formulas:
        print(i)
        
            
        if i[2] == '/':
            try:
                resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)/float(resulting_sheet.cell(column=i[3],row=4).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=4).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)/float(resulting_sheet.cell(column=i[3],row=5).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=5).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)/float(resulting_sheet.cell(column=i[3],row=6).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=6).value = '-'
            try:
                resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)/float(resulting_sheet.cell(column=i[3],row=7).value)                
            except ZeroDivisionError:
                resulting_sheet.cell(column=i[0],row=7).value = '-'

        if i[2] == '+':
            resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)+float(resulting_sheet.cell(column=i[3],row=4).value)+(resulting_sheet.cell(column=i[4],row=4).value)
            resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)+float(resulting_sheet.cell(column=i[3],row=5).value)+(resulting_sheet.cell(column=i[4],row=5).value)
            resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)+float(resulting_sheet.cell(column=i[3],row=6).value)+(resulting_sheet.cell(column=i[4],row=6).value)
            resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)+float(resulting_sheet.cell(column=i[3],row=7).value)+(resulting_sheet.cell(column=i[4],row=7).value)

        if i[2] == '-':
            resulting_sheet.cell(column=i[0],row=4).value = float(resulting_sheet.cell(column=i[1],row=4).value)-float(resulting_sheet.cell(column=i[3],row=4).value)
            resulting_sheet.cell(column=i[0],row=5).value = float(resulting_sheet.cell(column=i[1],row=5).value)-float(resulting_sheet.cell(column=i[3],row=5).value)
            resulting_sheet.cell(column=i[0],row=6).value = float(resulting_sheet.cell(column=i[1],row=6).value)-float(resulting_sheet.cell(column=i[3],row=6).value)
            resulting_sheet.cell(column=i[0],row=7).value = float(resulting_sheet.cell(column=i[1],row=7).value)-float(resulting_sheet.cell(column=i[3],row=7).value)
    wb_result.save('weekly_report.xlsx')
    #wb_test.save('weekly_report.xlsx')

    

#Тип объекта РО - запланировано по ППР
#Тип объекта ОТКЛОНЁН - ФАКТ, но необходимо смотреть на таблицу со сдвигом +102
#TODO добавить фильтр по РО

#Фактические отгрузки за одну неделю
#week - за какую неделю факт отгрузок
#type_of_transport - способ отгрузок(спобобы отгрузок в senders)
#работает и с накопительно (nakop_dates)
#senders ['all', 'auto', 'wagon', 'container']
def orion_by_mark(week, type_of_transport, fact_or_plan):
        if fact_or_plan == 'fact':
            f = 'ОТКЛОНЕНИЕ'
        if fact_or_plan == 'plan':
            f = 'РО'
        if type_of_transport == 'train':
            type_filtr = ''
        RESULT_BY_DAY = {}
        RESULT_SUM_BY_WEEK = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
        cur_m = week[0].month
        for date in week:
            if date.month == cur_m:
                orion = orion_df
            if date.month > cur_m:
                orion = orion_next_df
            cols = orion.columns.to_list()
            RESULT = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
            if fact_or_plan == 'fact':
                ind = cols.index(date) + 102
            if fact_or_plan == 'plan':
                ind = cols.index(date)
            
            for MARK in MARKS:
                if type_of_transport == 'auto':
                    FACT = orion[(orion['Тип объекта'] == f)\
                             & ((orion['Наименование   позиции'].str.contains(MARK) == True))\
                             & (orion['Метод отгрузки'] == 'Автотранспорт')].iloc[:,ind].sum()
                if type_of_transport == 'wagon':
                    FACT = orion[(orion['Тип объекта'] == f)\
                             & ((orion['Наименование   позиции'].str.contains(MARK) == True))\
                             & (orion['Метод отгрузки'] == 'ЖД -вагон')].iloc[:,ind].sum()
                if type_of_transport == 'container':
                    FACT = orion[(orion['Тип объекта'] == f)\
                             & ((orion['Наименование   позиции'].str.contains(MARK) == True))\
                             & ((orion['Метод отгрузки'] == 'ЖД -конт.')
                             |  (orion['Метод отгрузки'] == 'Авто конт.'))].iloc[:,ind].sum()
                if type_of_transport == 'all':
                    FACT = orion[(orion['Тип объекта'] == f)\
                             & ((orion['Наименование   позиции'].str.contains(MARK) == True))].iloc[:,ind].sum()
                if MARK == 'ДССК':
                    RESULT['СКД'] += FACT
                    RESULT_SUM_BY_WEEK['СКД'] += FACT
                else:
                    RESULT[MARK] += FACT
                    RESULT_SUM_BY_WEEK[MARK] += FACT
                
                RESULT_BY_DAY.update({date:RESULT})
        return RESULT_SUM_BY_WEEK
    
#фактическая отправка грузов в контейнера(как разность ВСЕГО_ОТПРАВЛЕНО - АВТО - ВАГОНЫ)
#на всякий случай
#depricated
def fact_get_container(week):
    all_types  = fact_by_mark(week, 'all')#{'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    print(all_types)
    wagons = fact_by_mark(week, 'wagon')
    print(wagons)
    autos = fact_by_mark(week, 'auto')
    print(autos)
    containers = {'СКС': int(all_types['СКС'] - wagons['СКС'] - autos['СКС']),\
                  'СКД': int(all_types['СКД'] - wagons['СКД'] - autos['СКД']),\
                  'ДСТ': int(all_types['ДСТ'] - wagons['ДСТ'] - autos['ДСТ']),\
                  'СБС': int(all_types['СБС'] - wagons['СБС'] - autos['СБС'])}# all_types - wagons - autos
    return containers


#VL06_TRANSPORT_TYPES = {'auto': 'ZAUT', 'wagon': 'ZRLC', 'container': '?'}
#УЗнать идентификатор контейнера
def vl06_by_mark(week, type_of_transport):
    RESULT  = {'СКС':0,'СКД':0,'ДСТ':0,'СБС':0}
    
    for MARK in MARKS:
        if type_of_transport == 'all':
            FACT = VL06[(VL06['Дата Накладной'] >= week[0])\
                             & (VL06['Дата Накладной'] <= week[-1])\
                             & (VL06['Название'].str.contains(MARK) == True)\
                             & (VL06['Завод'] == VSK_ID)]
        else:
            if type_of_transport == 'wagon':
                FACT = VL06[(VL06['Дата Накладной'] >= week[0])\
                             & (VL06['Дата Накладной'] <= week[-1])\
                             & (VL06['Название'].str.contains(MARK) == True)\
                             & (VL06['Завод'] == VSK_ID)
                             & ((VL06['Вид ТранспСредства'] == VL06_TRANSPORT_TYPES[type_of_transport])\
                             | (VL06['Ид. ТранспСредства'].str.isdecimal() == True))]
            if type_of_transport == 'auto':
                FACT = VL06[(VL06['Дата Накладной'] >= week[0])\
                             & (VL06['Дата Накладной'] <= week[-1])\
                             & (VL06['Название'].str.contains(MARK) == True)\
                             & (VL06['Завод'] == VSK_ID)
                             & ((VL06['Вид ТранспСредства'] == VL06_TRANSPORT_TYPES[type_of_transport])\
                             | (VL06['Ид. ТранспСредства'].str.isdecimal() == False))]
#                             Узнать идентификатор контейнера в сап!
#!!!!!!!!!!!!!!!!!!!!ИДЕНТИФИКАТОР!!!!!!!!!!!!!!!!!!!
            if type_of_transport == 'container':
                FACT = VL06[(VL06['Дата Накладной'] >= week[0])\
                             & (VL06['Дата Накладной'] <= week[-1])\
                             & (VL06['Название'].str.contains(MARK) == True)\
                             & (VL06['Завод'] == VSK_ID)
                             & ((VL06['Вид ТранспСредства'] == VL06_TRANSPORT_TYPES[type_of_transport])\
                             | (VL06['Ид. ТранспСредства'].str.isdecimal() == False))]
                
        #FACT.to_excel(MARK+'_'+type_of_transport+'.xlsx')
        if MARK == 'ДССК':
            RESULT['СКД'] += FACT['Объем поставки'].sum()
        else:
            RESULT[MARK] += FACT['Объем поставки'].sum()
    return RESULT


def get_8_and_9():
    main_info = main[main['Наименование предприятия'] == 'ВОРОНЕЖСИНТЕЗКАУЧУК']
    warehouse_max = {}
    vsk_productivity_max = {}

    counter = 0
    for m in MARKS[:4]:
        warehouse_max.update({m:main_info[['Максимальная вместимость склада, т']].iloc[counter,0]})
        vsk_productivity_max.update({m:main_info[['Максимальный объем производства']].iloc[counter,0]})
        counter+=1
    return warehouse_max, vsk_productivity_max

def get_sum(dict_var):
    tmp = dict_var.keys()
    summary = 0
    for i in tmp:
        summary += dict_var[i]
    return summary

In [ ]:
#Основной список вызов функций (main)
wb_result = create_resulting_wb()
result_sheet = wb_result['Отчёт']
copy_cells(report_titles, result_sheet)
copy_cells(hiden_data, result_sheet)
#orion_by_mark(week, type_of_sender, fact_or_plan):
#8,9
warehouse_max, productivity_max = get_8_and_9()
put_info_in_col(8, warehouse_max, result_sheet)
put_info_in_col(9, productivity_max, result_sheet)
#10
put_info_in_col(10, get_mb52(), result_sheet)
#12
put_info_in_col(12, orion_by_mark(next_week, 'all' , 'plan'), result_sheet)
#13
put_info_in_col(13, orion_by_mark(next_week, 'auto', 'plan'), result_sheet)
#14
put_info_in_col(14, orion_by_mark(next_week, 'container', 'plan'), result_sheet)
#15
put_info_in_col(15, orion_by_mark(next_week, 'wagon', 'plan'), result_sheet)
#16
put_info_in_col(16, {'СКС':1350,'СКД':3002,'ДСТ':689,'СБС':636}, result_sheet)
#17 столбец
put_info_in_col(17,get_column_17_info(prod_report),result_sheet)
#22
put_info_in_col(22, orion_by_mark(nakop_dates, 'all', 'plan'), result_sheet)
#23
#put_info_in_col(23, vl06_by_mark(nakop_dates, 'all'), result_sheet)
put_info_in_col(23, orion_by_mark(nakop_dates, 'all', 'fact'), result_sheet)
#25
put_info_in_col(25, orion_by_mark(cur_week, 'container', 'plan'), result_sheet)
#26
#put_info_in_col(26, vl06_by_mark(cur_week, 'container'), result_sheet)
put_info_in_col(26, orion_by_mark(cur_week, 'container', 'fact'), result_sheet)
#28
put_info_in_col(28, orion_by_mark(cur_week, 'wagon', 'plan'), result_sheet)
#29
#put_info_in_col(29, vl06_by_mark(cur_week, 'wagon'), result_sheet)
put_info_in_col(29, orion_by_mark(cur_week, 'wagon', 'fact'), result_sheet)
#31
put_info_in_col(31, orion_by_mark(cur_week, 'auto', 'plan'), result_sheet)
#32
#put_info_in_col(32, vl06_by_mark(cur_week, 'auto'), result_sheet)
put_info_in_col(32, orion_by_mark(cur_week, 'auto', 'fact'), result_sheet)
#предпоследняя
create_formulas_cols(result_sheet)
#последняя строка
find_total(result_sheet)
copy_cells(rows['total'], result_sheet)

In [ ]:
# #VL06 версия
# wb_result = create_resulting_wb()
# result_sheet = wb_result['Отчёт']
# copy_cells(report_titles, result_sheet)
# copy_cells(hiden_data, result_sheet)
# #orion_by_mark(week, type_of_sender, fact_or_plan):
# #8,9
# warehouse_max, productivity_max = get_8_and_9()
# put_info_in_col(8, warehouse_max, result_sheet)
# put_info_in_col(9, productivity_max, result_sheet)
# #10
# put_info_in_col(10, get_column10_info(mb52_report)[0], result_sheet)
# #12
# put_info_in_col(12, orion_by_mark(next_week, 'all' , 'plan'), result_sheet)
# #13
# put_info_in_col(13, orion_by_mark(next_week, 'auto', 'plan'), result_sheet)
# #14
# put_info_in_col(14, orion_by_mark(next_week, 'container', 'plan'), result_sheet)
# #15
# put_info_in_col(15, orion_by_mark(next_week, 'wagon', 'plan'), result_sheet)
# #16
# put_info_in_col(16, {'СКС':1350,'СКД':3002,'ДСТ':689,'СБС':636}, result_sheet)
# #17 столбец
# put_info_in_col(17,get_column_17_info(prod_report),result_sheet)
# #22
# put_info_in_col(22, orion_by_mark(nakop_dates, 'all', 'plan'), result_sheet)
# #23
# put_info_in_col(23, vl06_by_mark(nakop_dates, 'all'), result_sheet)
# #25
# put_info_in_col(25, orion_by_mark(cur_week, 'container', 'plan'), result_sheet)
# #26
# put_info_in_col(26, vl06_by_mark(cur_week, 'container'), result_sheet)
# #28
# put_info_in_col(28, orion_by_mark(cur_week, 'wagon', 'plan'), result_sheet)
# #29
# put_info_in_col(29, vl06_by_mark(cur_week, 'wagon'), result_sheet)
# #31
# put_info_in_col(31, orion_by_mark(cur_week, 'auto', 'plan'), result_sheet)
# #32
# put_info_in_col(32, orion_by_mark(cur_week, 'auto', 'fact'), result_sheet)
# #предпоследняя
# create_formulas_cols(result_sheet)
# #последняя строка
# find_total(result_sheet)
# copy_cells(rows['total'], result_sheet)

In [ ]:
# 'Продукт:'+str(RESULT_CLEAR), 'БРАК:'+ str(RESULT_BRAK611), 'ЭП:'+str(RESULT_EP)
clear, brak, ep = get_column10_info(mb52_report)
print(clear, brak, ep)
x1 = {'СКС':clear['СКС'] + brak['СКС'], 'СКД':clear['СКД'] + brak['СКД'], 'ДСТ':clear['ДСТ'] + brak['ДСТ'], 'СБС':clear['СБС'] + brak['СБС']}
x2 = {'СКС':clear['СКС'] + brak['СКС'] + ep['СКС'], 'СКД':clear['СКД'] + brak['СКД'] + ep['СКД'], 'ДСТ':clear['ДСТ'] + brak['ДСТ'] + ep['ДСТ'], 'СБС':clear['СБС'] + brak['СБС'] + ep['СБС']}
print(x1)
x2

In [ ]:
get_mb52()

In [ ]:
#get_column10_info(mb52_report)[1]
asd = 0
for i in range(4):
    asd+=zxc[i]
asd

In [ ]:
get_column10_info(mb52_report)[2]